### 网络结构

#### LRP

In [5]:
def LPR(x, index):   # [B, 3, N]
    '''
    输入:
        x: [B, 3, N]
        index: [B, N, k]
    返回:
        knn_x, relative_x, local_rep, exp_x, ratio_volume   # [B, 3, N, k]  [B, 3, N, k]   [B, 3, N, k]  [B, 1, N, k]  [B, N]
    '''
    B, _, N = x.shape
    k = index.shape[-1]

    knn_x = index_points(x, index)   # [B, 3, N, k]
    central_x = x.unsqueeze(-1).repeat(1, 1, 1, k)   # [B, 3, N, k]

    relative_x = central_x - knn_x   # [B, 3, N, k]

    # 相对点的 alpha 和 beta
    alpha = torch.atan2(relative_x[:, 1, :, :], relative_x[:, 0, :, :]).reshape(B, 1, N, k)   # [B, 1, N, k]
    xy_dist = torch.sqrt(torch.sum(torch.square(relative_x[:, :2, :, :]), dim=1)).reshape(B, 1, N, k)   # [B, 1, N, k]
    beta = torch.atan2(relative_x[:, 2:, :, :], xy_dist).reshape(B, 1, N, k)   # [B, 1, N, k]
    relative_dist = torch.sqrt(torch.sum(torch.square(relative_x), dim=1)).reshape(B, 1, N, k)   # [B, 1, N, k]

    exp_x = torch.exp(-relative_dist)   # [B, 1, N, k]
    local_volume = torch.pow(torch.max(relative_dist.squeeze(1), dim=-1)[0], 3)   # [B, N]  每个neighbor的体积

    # 质心点的 alpha 和 beta
    barycentrer_x = torch.mean(knn_x, dim=-1)   # [B, 3, N]  质心
    direction_relative = x - barycentrer_x   # [B, 3, N]
    direction_relative = direction_relative.unsqueeze(-1).repeat(1, 1, 1, k)   # [B, 3, N, k]
    barycentrer_alpha = torch.atan2(direction_relative[:, 1, :, :], direction_relative[:, 0, :, :]).reshape(B, 1, N, k)   # [B, 1, N, k]
    xy_dist = torch.sqrt(torch.sum(torch.square(direction_relative[:, :2, :, :]), dim=1)).reshape(B, 1, N, k)   # [B, 1, N, k]
    barycentrer_beta = torch.atan2(direction_relative[:, 2:, :, :], xy_dist).reshape(B, 1, N, k)   # [B, 1, N, k]

    angle = torch.cat([alpha-barycentrer_alpha, beta-barycentrer_beta], dim=1)   # [B, 2, N, k]

    local_rep = torch.cat([relative_dist, angle], dim=1)   # [B, 3, N, k]

    global_dist = torch.sqrt(torch.sum(torch.square(x), dim=1))   # [B, N]
    global_volume = torch.pow(torch.max(global_dist, dim=-1, keepdim=True)[0], 3)   # [B, 1]

    ratio_volume = local_volume / global_volume   # [B, N]

    return knn_x, relative_x, local_rep, exp_x, ratio_volume   # [B, 3, N, k]  [B, 3, N, k]   [B, 3, N, k]  [B, 1, N, k]  [B, N]

#### FlaResMLP

In [10]:
class InvResMLP(nn.Module):
    '''
    def __init__(self, input_channel):
    
    def forward(self, x, neighbor_index, feature):   # [B, 3, N]  [B, N, k]  [B, C, N]
    
    返回:
        feature    # [B, C, N]
    '''
    def __init__(self, input_channel):
        super().__init__()
        
        self.input_channel = input_channel
        
        self.mlpf = nn.Sequential(
                        nn.Conv1d(input_channel, (input_channel//3)*3, 1, bias=False),
                        nn.BatchNorm1d((input_channel//3)*3),
                        nn.LeakyReLU(0.2, True)
                    )
        
        self.mlp1 = nn.Sequential(
                       nn.Conv1d((input_channel//3)*3, 4*input_channel, 1, bias=False),
                       nn.BatchNorm1d(4*input_channel),
                       nn.ReLU(True)
                    )
        
        self.mlp2 = nn.Sequential(
                       nn.Conv1d(4*input_channel, input_channel, 1, bias=False),
                       nn.BatchNorm1d(input_channel),
                    )
    
    def forward(self, relative_x, x_encoding, neighbor_index, feature):   # [B, 3, N, k]  [B, C, N, k]  [B, N, k]  [B, C, N]
        identity = feature

        feature = self.mlpf(feature)   # [B, C, N]
        knn_feature = index_points(feature, neighbor_index)   # [B, C, N, k]
        feature = x_encoding + knn_feature   # [B, C, N, k]
        
        B, C, N, k = feature.shape
        
        x_en = relative_x.unsqueeze(2).repeat(1, 1, self.input_channel//3, 1, 1).reshape(B, C, N, k)
        feature = x_en * feature   # [B, C, N, k]
        
        feature = torch.max(feature, dim=-1)[0]   # [B, C, N]
        
        feature = self.mlp2(self.mlp1(feature))   # [B, C, N]
        
        # residual
        feature =  F.relu(feature+identity, inplace=True)   # [B, C, N]
        
        return feature    # [B, C, N]

#### Encoder

In [11]:
class Encoder(nn.Module):
    '''
    def __init__(self, input_channel, output_channel, k, stride, inverse_num):
    
    def forward(self, x, feature):   # [B, 3, N]  [B, C, N]
    
    return:
        feature, sub_x, neighbor_index   # [B, out, S]  [B, 3, S]
    '''
    def __init__(self, input_channel, output_channel, k, stride, inverse_num):
        super().__init__()
        
        self.k = k
        self.stride = stride

        self.mlp_f1 = nn.Sequential(
                        nn.Conv2d(input_channel, 1, 1),
                    )
        
        self.mlp_f2 = nn.Sequential(
                        nn.Conv2d(input_channel, output_channel//2, 1),
                        nn.BatchNorm2d(output_channel//2),
                        nn.ReLU(True)
                    )

        self.Rw = nn.Parameter(torch.randn(output_channel//2, 2))
        
        self.mlp1 = nn.Sequential(
                        nn.Conv2d(input_channel+9, output_channel, 1),
                        nn.BatchNorm2d(output_channel),
                        nn.ReLU(True)
                    )
        
        self.mlp2 = nn.Sequential(
                        nn.Conv1d(output_channel, output_channel, 1),
                        nn.BatchNorm1d(output_channel),
                    )
        self.res = nn.Sequential(
                        nn.Conv1d(input_channel, output_channel, 1),
                        nn.BatchNorm1d(output_channel),
                    )

        self.x_mlp = nn.Sequential(
                        nn.Conv2d(3, (output_channel//3)*3, 1),
                        nn.BatchNorm2d((output_channel//3)*3),
                        nn.ReLU(True)
                    )
        
        self.invres = nn.ModuleList()
        for i in range(inverse_num):
            self.invres.append(InvResMLP(output_channel))
        
        self.knn = KNN(k=k, transpose_mode=True)
        
    def forward(self, x, feature):   # [B, 3, N]  [B, C, N]
        _, _, N = x.shape
        
        identity = feature   # [B, C, N]
        
        _, neighbor_index = self.knn(x.permute(0, 2, 1).contiguous(), x.permute(0, 2, 1).contiguous())  # [B, N, k]

        # grouping
        knn_x, relative_x, local_rep, _, _ = LPR(x, neighbor_index)   # [B, 3, N, k]  [B, 3, N, k]  [B, 3, N, k]
        knn_feature = index_points(feature, neighbor_index) - feature.unsqueeze(-1)   # [B, C, N, k]
        
        x_enc = torch.cat([relative_x, local_rep], dim=1)   # [B, 6, N, k]

        scaling = self.mlp_f1(knn_feature)   # [B, 1, N, k]
        feature = self.mlp_f2(knn_feature)   # [B, out//2, N, k]
        rotation = (feature.permute(0, 2, 3, 1) @ self.Rw).permute(0, 3, 1, 2)   # [B, 2, N, k]

        feature = torch.cat([x_enc, knn_feature, scaling, rotation], dim=1)   # [B, C+9, N, k]
        feature = self.mlp1(feature)   # [B, out, N, k]
        feature = torch.max(feature, dim=-1)[0]   # [B, out, N]

        # residual
        feature = self.mlp2(feature)   # [B, out, N]
        res = self.res(identity)   # [B, out, N]
        feature = F.relu(feature+res, True)

        x_encoding = self.x_mlp(relative_x)
        for res in self.invres:
            feature = res(relative_x, x_encoding, neighbor_index, feature)

        # sub-sample
        sample_num = N//self.stride
        fps_idx = fps(x.permute(0, 2, 1).contiguous(), sample_num)
        sub_x = pointnet2_utils.gather_operation(x, fps_idx)   # [B, 3, S]
        sub_feature = pointnet2_utils.gather_operation(feature, fps_idx)    # [B, C, S]
        
        return sub_feature, sub_x, neighbor_index   # [B, out, S]  [B, 3, S]

#### serialization

In [12]:
def serialization(pos, feat, x_res=None, order="z", layers_outputs=[], grid_size=0.02):
    '''
    Args:
        order(str): [ "xyz", "xzy", "yxz", "yzx", "zxy", "zyx", "hilbert", "z", "z-trans" ]
    '''
    bs, n_p, _ = pos.size()
    if not isinstance(order, list):
        order = [order]

    scaled_coord = pos / grid_size
    grid_coord = torch.floor(scaled_coord).to(torch.int64)
    min_coord = grid_coord.min(dim=1, keepdim=True)[0]
    grid_coord = grid_coord - min_coord

    batch_idx = torch.arange(0, pos.shape[0], 1.0).unsqueeze(1).repeat(1, pos.shape[1]).to(torch.int64).to(pos.device)

    point_dict = {'batch': batch_idx.flatten(), 'grid_coord': grid_coord.flatten(0, 1), }
    point_dict = Point(**point_dict)
    point_dict.serialization(order=order)

    order = point_dict.serialized_order
    inverse_order = point_dict.serialized_inverse

    pos = pos.flatten(0, 1)[order].reshape(bs, n_p, -1).contiguous()
    feat = feat.flatten(0, 1)[order].reshape(bs, n_p, -1).contiguous()
    if x_res is not None:
        x_res = x_res.flatten(0, 1)[order].reshape(bs, n_p, -1).contiguous()

    for i in range(len(layers_outputs)):
        layers_outputs[i] = layers_outputs[i].flatten(0, 1)[order].reshape(bs, n_p, -1).contiguous()
    return pos, feat, x_res

#### Mamba Block

In [13]:
from typing import Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

try:
    from mamba_ssm.ops.triton.layernorm import RMSNorm, layer_norm_fn, rms_norm_fn
except ImportError:
    RMSNorm, layer_norm_fn, rms_norm_fn = None, None, None
from timm.models.layers import DropPath

class Block(nn.Module):
    def __init__(
        self, dim, mixer_cls, norm_cls=nn.LayerNorm, fused_add_norm=False,
            residual_in_fp32=False, drop_path=0.
    ):
        """
        Simple block wrapping a mixer class with LayerNorm/RMSNorm and residual connection"

        This Block has a slightly different structure compared to a regular
        prenorm Transformer block.
        The standard block is: LN -> MHA/MLP -> Add.
        [Ref: https://arxiv.org/abs/2002.04745]
        Here we have: Add -> LN -> Mixer, returning both
        the hidden_states (output of the mixer) and the residual.
        This is purely for performance reasons, as we can fuse add and LayerNorm.
        The residual needs to be provided (except for the very first block).
        """
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32
        self.fused_add_norm = fused_add_norm
        self.mixer = mixer_cls(dim)
        self.norm = norm_cls(dim)
        
        # drop path 
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        if self.fused_add_norm:
            assert RMSNorm is not None, "RMSNorm import fails"
            assert isinstance(
                self.norm, (nn.LayerNorm, RMSNorm)
            ), "Only LayerNorm and RMSNorm are supported for fused_add_norm"

    def forward(
        self, hidden_states: Tensor, residual: Optional[Tensor] = None, inference_params=None
    ):
        r"""Pass the input through the encoder layer.

        Args:
            hidden_states: the sequence to the encoder layer (required).
            residual: hidden_states = Mixer(LN(residual))
        """
        if not self.fused_add_norm:
            residual = (self.drop_path(hidden_states) + residual) if residual is not None else hidden_states
            hidden_states = self.norm(residual.to(dtype=self.norm.weight.dtype))
            if self.residual_in_fp32:
                residual = residual.to(torch.float32)
        else:
            fused_add_norm_fn = rms_norm_fn if isinstance(self.norm, RMSNorm) else layer_norm_fn
            hidden_states, residual = fused_add_norm_fn(
                self.drop_path(hidden_states),
                self.norm.weight,
                self.norm.bias,
                residual=residual,
                prenorm=True,
                residual_in_fp32=self.residual_in_fp32,
                eps=self.norm.eps,
            )
        hidden_states = self.mixer(hidden_states, inference_params=inference_params)
        return hidden_states, residual

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return self.mixer.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)

In [14]:
from functools import partial

from mamba_ssm.modules.mamba_simple import Mamba

def create_block(
        d_model,
        ssm_cfg=None,
        norm_epsilon=1e-5,
        rms_norm=False,
        residual_in_fp32=False,
        fused_add_norm=False,
        layer_idx=None,
        drop_path=0.,
        device=None,
        dtype=None,
):
    if ssm_cfg is None:
        ssm_cfg = {}
    factory_kwargs = {"device": device, "dtype": dtype}

    mixer_cls = partial(Mamba, layer_idx=layer_idx, **ssm_cfg, **factory_kwargs)
    norm_cls = partial(
        nn.LayerNorm if not rms_norm else RMSNorm, eps=norm_epsilon, **factory_kwargs
    )
    block = Block(
        d_model,
        mixer_cls,
        norm_cls=norm_cls,
        fused_add_norm=fused_add_norm,
        residual_in_fp32=residual_in_fp32,
        drop_path=drop_path,
    )
    block.layer_idx = layer_idx
    return block

#### Mamba

In [15]:
def _init_weights(
        module,
        n_layer,
        initializer_range=0.02,  # Now only used for embedding layer.
        rescale_prenorm_residual=True,
        n_residuals_per_layer=1,  # Change to 2 if we have MLP
):
    if isinstance(module, nn.Linear):
        if module.bias is not None:
            if not getattr(module.bias, "_no_reinit", False):
                nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        nn.init.normal_(module.weight, std=initializer_range)

    if rescale_prenorm_residual:
        # Reinitialize selected weights subject to the OpenAI GPT-2 Paper Scheme:
        #   > A modified initialization which accounts for the accumulation on the residual path with model depth. Scale
        #   > the weights of residual layers at initialization by a factor of 1/√N where N is the # of residual layers.
        #   >   -- GPT-2 :: https://openai.com/blog/better-language-models/
        #
        # Reference (Megatron-LM): https://github.com/NVIDIA/Megatron-LM/blob/main/megatron/model/gpt_model.py
        for name, p in module.named_parameters():
            if name in ["out_proj.weight", "fc2.weight"]:
                # Special Scaled Initialization --> There are 2 Layer Norms per Transformer Block
                # Following Pytorch init, except scale by 1/sqrt(2 * n_layer)
                # We need to reinit p since this code could be called multiple times
                # Having just p *= scale would repeatedly scale it down
                nn.init.kaiming_uniform_(p, a=math.sqrt(5))
                with torch.no_grad():
                    p /= math.sqrt(n_residuals_per_layer * n_layer)


In [16]:
import math

class MixerModel(nn.Module):
    def __init__(
            self,
            d_model: int,
            n_layer: int,
            ssm_cfg=None,
            norm_epsilon: float = 1e-5,
            rms_norm: bool = False,
            initializer_cfg=None,
            fused_add_norm=False,
            residual_in_fp32=False,
            drop_out_in_block: int = 0.,
            drop_path: int = 0.1,
            device=None,
            dtype=None,
    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()
        self.residual_in_fp32 = residual_in_fp32

        # self.embedding = nn.Embedding(vocab_size, d_model, **factory_kwargs)

        # We change the order of residual and layer norm:
        # Instead of LN -> Attn / MLP -> Add, we do:
        # Add -> LN -> Attn / MLP / Mixer, returning both the residual branch (output of Add) and
        # the main branch (output of MLP / Mixer). The model definition is unchanged.
        # This is for performance reason: we can fuse add + layer_norm.
        self.fused_add_norm = fused_add_norm
        if self.fused_add_norm:
            if layer_norm_fn is None or rms_norm_fn is None:
                raise ImportError("Failed to import Triton LayerNorm / RMSNorm kernels")

        self.layers = nn.ModuleList(
            [
                create_block(
                    d_model,
                    ssm_cfg=ssm_cfg,
                    norm_epsilon=norm_epsilon,
                    rms_norm=rms_norm,
                    residual_in_fp32=residual_in_fp32,
                    fused_add_norm=fused_add_norm,
                    layer_idx=i,
                    drop_path=drop_path,
                    **factory_kwargs,
                )
                for i in range(n_layer)
            ]
        )

        self.norm_f = (nn.LayerNorm if not rms_norm else RMSNorm)(
            d_model, eps=norm_epsilon, **factory_kwargs
        )

        self.apply(
            partial(
                _init_weights,
                n_layer=n_layer,
                **(initializer_cfg if initializer_cfg is not None else {}),
            )
        )
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.drop_out_in_block = nn.Dropout(drop_out_in_block) if drop_out_in_block > 0. else nn.Identity()

    def allocate_inference_cache(self, batch_size, max_seqlen, dtype=None, **kwargs):
        return {
            i: layer.allocate_inference_cache(batch_size, max_seqlen, dtype=dtype, **kwargs)
            for i, layer in enumerate(self.layers)
        }

    def forward(self, input_ids, pos, inference_params=None):
        hidden_states = input_ids  # + pos
        residual = None
        hidden_states = hidden_states + pos
        for layer in self.layers:                   # hidden_states: all 32, 192, 384
            hidden_states, residual = layer(
                hidden_states, residual, inference_params=inference_params
            )
            hidden_states = self.drop_out_in_block(hidden_states)
        if not self.fused_add_norm:
            residual = (hidden_states + residual) if residual is not None else hidden_states
            hidden_states = self.norm_f(residual.to(dtype=self.norm_f.weight.dtype))
        else:
            # Set prenorm=False here since we don't need the residual
            fused_add_norm_fn = rms_norm_fn if isinstance(self.norm_f, RMSNorm) else layer_norm_fn
            hidden_states = fused_add_norm_fn(
                hidden_states,
                self.norm_f.weight,
                self.norm_f.bias,
                eps=self.norm_f.eps,
                residual=residual,
                prenorm=False,
                residual_in_fp32=self.residual_in_fp32,
            )

        return hidden_states

#### FeaturePropagation

In [17]:
class FeaturePropagation(nn.Module):
    '''
    def __init__(self, input_channel, mlp)
    
    def forward(self, coords1, coords2, points1, points2)
        输入：
            某一层输入：coords1  [B, C, N], points1  [B, D, N]
            某一层输出：coords2  [B, C, S]，points2  [B, D', S]
        返回：
            new_points  [B, D'', N]
    '''
    def __init__(self, input_channel, mlp):
        super().__init__()
        self.mlp_conv = nn.ModuleList()
        self.mlp_bn = nn.ModuleList()
        for output_channel in mlp:
            self.mlp_conv.append(nn.Conv1d(input_channel, output_channel, 1, bias=False))
            self.mlp_bn.append(nn.BatchNorm1d(output_channel))
            input_channel = output_channel

    def forward(self, coords1, coords2, points1, points2):
        '''                                                     第一层          第二层          第三层          第四层
            coords1：[B, C, N]   Set Abstraction的输入     eg.  [B, 3, 2048]    [B, 3, 1024]    [B, 3, 256]     [B, 3, 64]
            coords2：[B, C, S]   Set Abstraction的输出     eg.  [B, 3, 1024]    [B, 3, 256]     [B, 3, 64]      [B, 3, 16]
            points1：[B, D, N]   Set Abstraction的输入     eg.  [B, 9, 2048]    [B, 64, 1024]   [B, 128, 256]   [B, 256, 64]
            points2：[B, D', S]  Set Abstraction的输出     eg.  [B, 64, 1024]   [B, 128, 256]   [B, 256, 64]    [B, 512, 16]
        '''
        # 以第四层为例标明注释
        
        coords1 = coords1.permute(0, 2, 1)    # [B, 64, 3]
        coords2 = coords2.permute(0, 2, 1)    # [B, 16, 3]
        
        dists, index = three_nn(coords1, coords2)   # [B, 64, 3]  [B, 16, 3]
        
        dist_recip = 1 / (dists + 1e-8)
        norm = torch.sum(dist_recip, -1, keepdim=True)
        weight = dist_recip / norm    # [B, 64, 3]
        
        interpolated_points = three_interpolate(points2, index, weight)

        new_points = torch.cat([points1, interpolated_points], dim=1)   # [B, 768, 64]

        for conv,bn in zip(self.mlp_conv, self.mlp_bn):
            new_points = F.relu(bn(conv(new_points)), inplace=True)
        # 最后输出new_points.shape: [B, 256, 64]
        
        return new_points

#### semseg network

In [19]:
class semseg_network(nn.Module):
    def __init__(self, seg_num=13, k=16, stride=[4, 4, 4, 2]):
        super().__init__()
        
        self.conv0 = nn.Sequential(
                         nn.Conv1d(7, 32, 1),
                         nn.BatchNorm1d(32),
                         nn.ReLU(inplace=True)
                     )
        
        self.encoder1 = Encoder(32, 64, k, stride[0], 3)
        self.encoder2 = Encoder(64, 128, k, stride[1], 6)
        self.encoder3 = Encoder(128, 256, k, stride[2], 3)
        self.encoder4 = Encoder(256, 512, k, stride[3], 3)

        self.mlp = nn.Sequential(
                       nn.Conv1d(512, 512, 1),
                       nn.BatchNorm1d(512),
                       nn.ReLU(True)
                   )

        self.fp4 = FeaturePropagation(768, [256])
        self.emb4 = nn.Sequential(
            nn.Linear(3, 128),
            nn.GELU(),
            nn.Linear(128, 256)
        )
        self.mamba4 = MixerModel(d_model=256, n_layer=2, rms_norm=False, drop_out_in_block=0, drop_path=0.3)
        
        self.fp3 = FeaturePropagation(384, [128])
        self.emb3 = nn.Sequential(
            nn.Linear(3, 128),
            nn.GELU(),
            nn.Linear(128, 128)
        )
        self.mamba3 = MixerModel(d_model=128, n_layer=2, rms_norm=False, drop_out_in_block=0, drop_path=0.3)
        
        self.fp2 = FeaturePropagation(192, [64])
        self.emb2 = nn.Sequential(
            nn.Linear(3, 128),
            nn.GELU(),
            nn.Linear(128, 64)
        )
        self.mamba2 = MixerModel(d_model=64, n_layer=2, rms_norm=False, drop_out_in_block=0, drop_path=0.3)


        
        self.fp1 = FeaturePropagation(96, [32])
        self.emb1 = nn.Sequential(
            nn.Linear(3, 128),
            nn.GELU(),
            nn.Linear(128, 32)
        )
        self.mamba1 = MixerModel(d_model=32, n_layer=2, rms_norm=False, drop_out_in_block=0, drop_path=0.3)
        
        self.segMLP = nn.Sequential(
                          nn.Conv1d(32, 32, 1),
                          nn.BatchNorm1d(32),
                          nn.ReLU(True),
                          nn.Conv1d(32, seg_num, 1)
                      )


    def forward(self, x, feature):

        feature0 = self.conv0(feature)   # [B, 32, N]

        feature1, x1, neighbor_index0 = self.encoder1(x, feature0)   # [B, 128, N//4]
        feature2, x2, neighbor_index1 = self.encoder2(x1, feature1)   # [B, 256, N//16]
        feature3, x3, neighbor_index2 = self.encoder3(x2, feature2)   # [B, 512, N//64]
        feature4, x4, neighbor_index3 = self.encoder4(x3, feature3)   # [B, 1024, N//128]

        feature4 = self.mlp(feature4)   # [B, 1024, N//512]

        feature3 = self.fp4(x3, x4, feature3, feature4)   # [B, 256, N//64]
        feature3 = self.mamba4(feature3.permute(0, 2, 1), self.emb4(x3.permute(0, 2, 1))).permute(0, 2, 1).contiguous()
        
        feature2 = self.fp3(x2, x3, feature2, feature3)   # [B, 128, N//16]
        feature2 = self.mamba3(feature2.permute(0, 2, 1), self.emb3(x2.permute(0, 2, 1))).permute(0, 2, 1).contiguous()
        
        feature1 = self.fp2(x1, x2, feature1, feature2)   # [B, 32, N//4]
        feature1 = self.mamba2(feature1.permute(0, 2, 1), self.emb2(x1.permute(0, 2, 1))).permute(0, 2, 1).contiguous()
        
        feature0 = self.fp1(x, x1, feature0, feature1)   # [B, 32, N]
        feature0 = self.mamba1(feature0.permute(0, 2, 1), self.emb1(x.permute(0, 2, 1))).permute(0, 2, 1).contiguous()

        # feature0 = torch.cat([feature3.max(-1)[0], feature2.max(-1)[0], feature1.max(-1)[0], feature0.max(-1)[0]], dim=1)

        result = self.segMLP(feature0)  # [B, seg_num, N]

        return result   # [B, seg_num, N]